# **Data Preprocessing**

This notebook performs the preprocessing of molecular data for QSAR modeling. The primary goals are to clean the dataset, remove potentially problematic compounds, and ensure the quality of descriptors for downstream machine learning workflows.

## **Step 1: Preparation of SMILES Files for QSAR Modeling**

The positive dataset consists of antioxidant compounds (**Positive**) sourced from `Antioxidant-Compound-Library-96-well.xlsx`. 
Since these compounds are also included within the broader `Bioactive-Compound-Library-I-96-well.xlsx`, we define the negative set by subtracting the positive compounds from the total bioactive library.

1. Load Excel files containing compound names and SMILES strings.
2. Remove duplicate and missing SMILES from each dataset.
3. Extract the **negative** set by removing overlapping SMILES with the **positive** set.
4. Save the datasets in `.smi` format for subsequent descriptor calculation using PaDEL.

### **Input and Output files:**
* Input files:
    * antioxi_data_817.smi (**Antioxidant: Positive**)
    * total_data_9972.smi (**Total bioactive compounds**) 
* Raw Outputs:
    * antioxi_data_802.smi (Positive compound SMILES format)
    * only_neg_data_8898.smi (Negative compound SMILES format)

#### **Converting raw files to smi files**

In [7]:
# import pandas as pd
# import os
# from tqdm import tqdm

# # Load Raw data
# antioxi_raw_data = pd.read_excel("20231011-L6500-Antioxidant-Compound-Library-96-well.xlsx", sheet_name=1, engine="openpyxl")
# total_raw_data = pd.read_excel("20240327-L1700-Bioactive-Compound-Library-I-96-well.xlsx", sheet_name=1, engine="openpyxl")

# # Preprocess data
# antioxi_data = antioxi_raw_data[["SMILES", 'Name']]                                                                                 # n = 817
# total_data = total_raw_data[["SMILES", 'Name']]                                                                                     # n = 9972

# # Save to SMI file
# antioxi_data.to_csv("antioxi_data_817.smi", sep='\t', header=False, index=False)  
# total_data.to_csv("total_data_9972.smi", sep='\t', header=False, index=False)    

In [17]:
# Load the SMI files
df_antioxi = pd.read_csv("antioxi_data_817.smi", sep="\t", header=None, names=["SMILES", "Name"])
df_total   = pd.read_csv("total_data_9972.smi", sep="\t", header=None, names=["SMILES", "Name"])

# Removal of duplication and NaN values
antioxi_data_dup = df_antioxi.drop_duplicates(['SMILES'], keep = 'first', ignore_index = True)                                      # n = 803
antioxi_data_nan = antioxi_data_dup.dropna(subset = ['SMILES'], ignore_index = True)                                                # n = 802

total_data_dup = df_total.drop_duplicates(['SMILES'], keep = 'first', ignore_index = True)                                          # n = 9701
total_data_nan = total_data_dup.dropna(subset = ['SMILES'], ignore_index = True)                                                    # n = 9700

# Merge the DataFrames to find the rows in total_data_nan not present in antioxi_data_nan
merged_data = pd.merge(total_data_nan, antioxi_data_nan, on="SMILES", how="left", indicator=True)

# Filter the merged DataFrame to keep only the rows that are in total_data_nan but not in antioxi_data_nan
only_neg_data = merged_data[merged_data['_merge'] == 'left_only']

# Drop the indicator column
only_neg_data = only_neg_data.drop(columns=['_merge'])

# Rename the columns: drop "Name_y" and rename "Name_x" to "Name"
only_neg_data = only_neg_data.drop(columns=['Name_y']).rename(columns={'Name_x': 'Name'})

# Remove spaces in the "Name" column
antioxi_data_nan.loc[:, "Name"] = antioxi_data_nan["Name"].str.replace(' ', '')
# Save to SMI file
antioxi_data_nan.to_csv('antioxi_data_802.smi', sep='\t', index=False, header=False)                                                # n = 802

# Remove spaces in the "Name" column
only_neg_data.loc[:, "Name"] = only_neg_data["Name"].str.replace(' ', '')
only_neg_data.to_csv("only_neg_data_8898.smi", sep='\t', header=False, index=False)                                                 # n = 8898

## **Step 2: Filter Out Metal-Containing Compounds**

Metal-containing compounds are often problematic in QSAR modeling. They can interfere with descriptor calculation tools such as PaDEL, introduce noise into machine learning models, and lead to poor generalization. To address this, we implemented a rule-based filtering strategy that removes any compound whose SMILES string contains known metal ions.

A list of common metal elements, including alkali metals, alkaline earth metals, transition metals, and heavy metals, was used as a matching reference. Additionally, placeholder tokens such as `[R]` and SMARTS-related annotations like `;v` were excluded.

### **Input and Output files:**
* Input files:
    * antioxi_data_802.smi
    * only_neg_data_8898.smi (Total **Negative** compounds)
* Filtered Outputs:
    * filtered_antioxi_data_776.smi
    * filtered_only_neg_data_8443.smi

In [13]:
# Define function to filter out metal ions
def filter_metal_ions(smiles_file, output_file):
    # 메탈 이온 목록
    metal_ions = ['Li', 'Be', 'Na', 'Mg', 'Al', 'K', 'Ca', 'Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn', 
                  'Ga', 'Ge', 'Rb', 'Sr', 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd', 'In', 'Sn', 
                  'Sb', 'Cs', 'Ba', 'La', 'Ce', 'Pr', 'Nd', 'Pm', 'Sm', 'Eu', 'Gd', 'Tb', 'Dy', 'Ho', 'Er', 'Tm', 
                  'Yb', 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg', 'Tl', 'Pb', 'Bi', 'Th', 'Pa', 'U','[R]',';v']

    with open(smiles_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            parts = line.strip().split('\t')                        
            if len(parts) == 2:
                smile, name = parts
                if not any(metal in smile for metal in metal_ions):
                    outfile.write(f"{smile}\t{name}\n")

# Filter metal ions from antioxi_data_nan
input_file_antioxi = 'antioxi_data_802.smi'
output_file_antioxi = 'filtered_antioxi_data_776.smi'                                                                               # n = 776
filter_metal_ions(input_file_antioxi, output_file_antioxi)

# Filter metal ions from total_data_nan
input_file_total = 'only_neg_data_8898.smi'
output_file_total = 'filtered_only_neg_data_8443.smi'                                                                               # n = 8443
filter_metal_ions(input_file_total, output_file_total)

## **Step 3: Calculation of 1,444 2D Molecular Descriptors using PaDELpy**

Molecular descriptors were computed using the `PaDELpy` Python wrapper for the PaDEL-Descriptor Java tool. 
A total of **1,444 2D descriptors** were calculated for each compound.

When the dataset was too large to process in a single run, which may cause memory issues or timeouts. 
To mitigate this, the .smi file was split into smaller chunks (each containing 500 compounds), and descriptors were computed separately for each chunk.

1. Calculate 1,444 2D molecular descriptors using PaDELpy
2. Remove compounds with duplicate entries and missing descriptor values (NaN).
3. Assign binary class labels: '1' for antioxidant (positive), and '0' for non-antioxidant (negative) compounds.

### **Input and Output files:**
* Input files:
    * filtered_antioxi_data_776.smi
    * filtered_only_neg_data_8443.smi
* Calculated Outputs:
    * antioxi_des_776.csv
    * negative_des_8443.csv
* Processed Outputs:
    * antioxidant_des_dupna_727.csv (Positive descriptors with label 1)
    * negative_des_dupna_6677.csv   (Negative descriptors with label 0)

In [1]:
### Positive set ###
from padelpy import padeldescriptor

padeldescriptor(mol_dir='filtered_antioxi_data_776.smi', d_file='antioxi_des_776.csv', 
                d_2d=True, d_3d=False, detectaromaticity=True, 
                log=True, removesalt=True, standardizenitro=True, 
                usefilenameasmolname=False, 
                retainorder=True, threads=-1, waitingjobs=-1, 
                maxruntime=10000, maxcpdperfile=0, headless=True)

In [ ]:
### Negative set ###
from padelpy import padeldescriptor

padeldescriptor(mol_dir='filtered_only_neg_data_8443.smi', d_file='negative_des_8443.csv', 
                d_2d=True, d_3d=False, detectaromaticity=True, 
                log=True, removesalt=True, standardizenitro=True, 
                usefilenameasmolname=False, 
                retainorder=True, threads=-1, waitingjobs=-1, 
                maxruntime=10000, maxcpdperfile=0, headless=True)

In [ ]:
### Negative set V2 (When an error occurs during descriptor calculation due to large data size) ###
# import os
# import pandas as pd
# from padelpy import padeldescriptor
# import glob
# from tqdm import tqdm

# def split_smi_file(input_file, output_dir, chunk_size):
#     with open(input_file, 'r') as file:
#         lines = file.readlines()
        
#     num_chunks = len(lines) // chunk_size + (1 if len(lines) % chunk_size != 0 else 0)
    
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
    
#     for i in range(num_chunks):
#         chunk_lines = lines[i * chunk_size: (i + 1) * chunk_size]
#         chunk_filename = os.path.join(output_dir, f'chunk_{i + 1}.smi')
#         with open(chunk_filename, 'w') as chunk_file:
#             chunk_file.writelines(chunk_lines)
    
#     return num_chunks

# # Example usage
# num_chunks = split_smi_file('filtered_only_neg_data_8443.smi', 'chunks', 500)
# print(f'Total {num_chunks} chunks created.')

# chunk_files = glob.glob('chunks/*.smi')

# for chunk_file in tqdm(chunk_files, desc="Calculating descriptors for each chunk"):
#     output_csv = chunk_file.replace('.smi', '.csv')
#     padeldescriptor(mol_dir=chunk_file, d_file=output_csv, 
#                     d_2d=True, d_3d=False, detectaromaticity=True, 
#                     log=True, removesalt=True, standardizenitro=True, 
#                     usefilenameasmolname=False, 
#                     retainorder=True, threads=16, waitingjobs=-1, 
#                     maxruntime=10000, maxcpdperfile=0, headless=True)


    

# csv_files = glob.glob('chunks/*.csv')

# dataframes = [pd.read_csv(csv_file) for csv_file in csv_files]
# merged_df = pd.concat(dataframes, ignore_index=True)

# merged_df.to_csv('negative_des_8443_merged.csv', index=False)

In [14]:
### Data loading & processing ###
positive = pd.read_csv("antioxi_des_776.csv", index_col='Name')                                                                        # n =776
negative = pd.read_csv("negative_des_8443.csv", index_col = "Name")                                                                    # n = 8443

neg_df_dup = negative.drop_duplicates(keep = 'first')                                                                                  # n = 8369
pos_df_dup = positive.drop_duplicates(keep='first')                                                                                    # n = 751
pos_df, neg_df = pos_df_dup.dropna(), neg_df_dup.dropna()                                                                
pos_df, neg_df = (pos_df.assign(y_true=1), neg_df.assign(y_true=0))                                                                    # n = 727, 6677       

neg_df.to_csv("negative_des_dupna_6677.csv", index=True)
pos_df.to_csv("antioxidant_des_dupna_727.csv", index=True)


## **Step 4: Train/Test Split (7:3 ratio)**

To prepare for model training and evaluation, the dataset was divided into training and test sets using a **7:3 split ratio** for the positive class. 
The negative class was subsampled to ensure a **balanced test set**.

1. **Positive compounds** were randomly split into:
   - 70% training set (n = 508)
   - 30% test set (n = 219)
2. **Negative compounds** were randomly sampled to match the number of positive test samples (n = 219), ensuring a **balanced test set**.
3. The remaining negative compounds (n = 6,458) were used in the training set alongside the positive training samples.
4. All sets were saved as CSV files with compound names as index and descriptor values plus a binary label (`y_true`).

### **Input and Output files:**
* Input File:
    * antioxidant_des_dupna_727.csv
    * negative_des_dupna_6677.csv
* Split Outputs:
    * ./train/train_pos_508.csv  (n = 508)  
    * ./train/train_total_neg_6458.csv
    * ./test/test_set_438.csv

In [17]:
### train : test = 7: 3 split (pos) ###
from sklearn.model_selection import train_test_split

# Load the preprocessed positive and negative descriptor datasets
# neg_df = pd.read_csv("negative_des_dupna_6677.csv", index_col='Name')
# pos_df = pd.read_csv("antioxidant_des_dupna_727.csv", index_col='Name')

# Split the positive set into 70% training and 30% test sets
train_pos, test_pos = train_test_split(pos_df, test_size=0.3, random_state=1004)                                                      # train_pos = 508, test_pos = 219

# Randomly sample an equal number of negatives to match the positive test set (219 samples)
test_neg = neg_df.sample(n=len(test_pos), random_state=1004)                                                                          # test_neg = 219
test_set = pd.concat([test_pos, test_neg])                                                                                            # test_set = 438

# Exclude selected negative test samples from the negative pool to define the negative training set
train_total_neg = neg_df.drop(test_neg.index)                                                                                         # train_neg = 6458 

# Save the resulting datasets to CSV files
os.makedirs("./train", exist_ok=True)
os.makedirs("./test", exist_ok=True)

train_pos.to_csv("./train/train_pos_508.csv", index=True)                                                                             
train_total_neg.to_csv("./train/train_total_neg_6458.csv", index=True)                                                               
#test_pos.to_csv("./test/test_pos_219.csv", index=True)                                                                                   
#test_neg.to_csv("./test/test_neg_219.csv", index=True)                                                                               
test_set.to_csv("./test/test_set_438.csv", index=True)   

## **Step 5: Descriptor Filtering by Low Variance**


To reduce model complexity and eliminate uninformative features, molecular descriptors with low variance across the dataset were removed.  
Features that exhibit minimal variation across both positive and negative classes are unlikely to contribute to classification performance, as they do not help differentiate the classes.

1. Combine positive and negative training sets vertically into a single DataFrame.
2. Remove the `y_true` label column for feature-only variance analysis.
3. Apply a variance threshold of **0.05**: descriptors with variance below this value were removed.
4. Construct a filtered training set based only on the selected descriptors.
5. Split the filtered set back into positive and negative groups.


In [18]:
from sklearn.feature_selection import VarianceThreshold

# train_pos = pd.read_csv("./train/train_pos_508.csv", index_col='Name')
# pos_df = pd.read_csv("antioxidant_des_dupna_727.csv", index_col='Name')

# Concatenate positive and total negative samples vertically
train_total_data = pd.concat([train_pos, train_total_neg], axis=0)

# Drop the target column 'y_true' to get feature matrix
X_train_total_data = train_total_data.drop(columns=['y_true'])
y_train_total_data = train_total_data['y_true']

# Remove low variance features - VarianceThreshold removes features with variance lower than the specified threshold
threshold = 0.05  # Example: Remove features with variance less than 5%
selector = VarianceThreshold(threshold=threshold)
X_low_variance_removed = selector.fit_transform(X_train_total_data)

# Get the names of selected features
selected_features = X_train_total_data.columns[selector.get_support()]
print("Number of features remaining after low variance removal:", len(selected_features))

# Create DataFrame with selected features
train_data_selected = pd.DataFrame(X_low_variance_removed, columns=selected_features)
train_data_selected['y_true'] = y_train_total_data.reset_index(drop=True)

# Split into positive and negative samples based on 'y_true' value
train_pos_selected = train_data_selected[train_data_selected['y_true'] == 1].copy()
train_neg_selected = train_data_selected[train_data_selected['y_true'] == 0].copy()

# Restore the original index from train_pos and train_total_neg
train_pos_selected.index = train_pos.index
train_neg_selected.index = train_total_neg.index

# Print the result
print("Positive sample shape:", train_pos_selected.shape)
print("Negative sample shape:", train_neg_selected.shape)


Low variance 제거 후 남은 feature 개수: 893
Positive sample shape: (508, 894)
Negative sample shape: (6458, 894)


# **Resampling for class balancing**

This section addresses the inherent class imbalance in the training data by applying a resampling strategy. 
Since the number of negative samples exceeds that of positive samples, we generate multiple balanced training sets by randomly sampling the negative class with replacement to match the size of the positive class.

## **Steps:**
1. Merge `train_pos_selected` and `train_neg_selected` after descriptor filtering.
2. Repeat the following process 300 times:
   - Sample 508 negative compounds **with replacement** from `train_neg_selected` using varying random seeds.
   - Combine with the 508 positive compounds to form a balanced binary classification dataset.
3. Export each combined training set (total 1016 compounds) to the `./train/` directory for subsequent modeling.
   
### **Output files:**
* Resampled Outputs:
    * ./train/train_set_(1~300)_1016.csv (1016 X 893)  

In [19]:
# Randomly sample negative data to match the number of positive samples (with replacement)
for i in range(300):
    train_neg = train_neg_selected.sample(n=len(train_pos_selected), replace=True, random_state=1004 + i)  # Varying random state per iteration
    train_set = pd.concat([train_pos_selected, train_neg])
    # train_neg.to_csv(f'./train/train_neg_{i+1}_508.csv', index=True)  # Optional: save each negative set (508 x 300)
    train_set.to_csv(f'./train/train_set_{i+1}_1016.csv', index=True)    # Save combined train set (1016 samples x 300 sets)
                            